In [1]:
%load_ext autoreload 
%autoreload 2

%aimport -pandas
%aimport -scipy.stats
%aimport -numpy
%aimport -matplotlib.pyplot


import default_calcs as default
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas
import collateral_waterfall as cw
import prepayment_calcs as pc

In [43]:
initial_balance = 100e6    # 100 million
net_coupon      = 0.10     # 10%
gross_coupon    = 0.1065   # 10.65%
maturity        = 360      # 30 years => monthly
servicing       = 0.0025   # servicing = 25bps

bonds = [
    {'Bond': 'A',
    'Balance': 30e6,
    'Coupon': 0.07},

    {'Bond': 'B',
    'Balance': 40e6,
    'Coupon': 0.09},
    
    {'Bond': 'C',
    'Balance': 30e6,
    'Coupon': 0.10}
]

In [ ]:
psa_100 = cw.create_waterfall(original_balance=initial_balance,
                             pass_thru_cpn=net_coupon,
                             wac=gross_coupon,
                             wam=maturity,
                             psa_speed=1,
                             servicing=servicing)
psa_175 = cw.create_waterfall(original_balance=initial_balance,
                             pass_thru_cpn=net_coupon,
                             wac=gross_coupon,
                             wam=maturity,
                             psa_speed=1.75,
                             servicing=servicing)
psa_400 = cw.create_waterfall(original_balance=initial_balance,
                             pass_thru_cpn=net_coupon,
                             wac=gross_coupon,
                             wam=maturity,
                             psa_speed=4,
                             servicing=servicing)

In [ ]:
print('PSA\tAverage Life')
print('100%:\t{0:.2f}'.format(weighted_average_life(psa_100)/12))
print('175%:\t{0:.2f}'.format(weighted_average_life(psa_175)/12))
print('400%:\t{0:.2f}'.format(weighted_average_life(psa_400)/12))


In [ ]:
%matplotlib
frames = [psa_100,psa_175,psa_400]

fig, axes = plt.subplots(3,2)
for i in range(3):
    for j in range(2):
        if j == 0:
            axes[i,j].fill_between(frames[i].index-1,
                                  frames[i].beginning_balance)
        else:
            axes[i,j].stackplot(frames[i].index-1, 
                          frames[i].scheduled_principal,
                          frames[i].prepayments,
                          frames[i].net_interest,
                          frames[i].servicing)
            
plt.tight_layout(True)
plt.show()

In [ ]:
def calc_seq_bond_cfs(collateral_waterfall, bonds):
    bond_waterfalls = {}
    for bond in bonds:
        current_bond = bond['Bond']
        bond_waterfalls[current_bond] = pd.DataFrame(
            index=psa_175.index.values,
            columns=['Bond_' + current_bond,
                    'Coupon_' + current_bond,
                    'Balance_' + current_bond,
                    'Principal_' + current_bond,
                    'Interest_' + current_bond,
                    'Cashflow_' + current_bond])
        bond_waterfalls[current_bond]['Bond_' + current_bond] = current_bond
        bond_waterfalls[current_bond]['Coupon_' + current_bond] = bond['Coupon']
        bond_waterfalls[current_bond].loc[1,'Balance_' + current_bond] = bond['Balance']
        bond_waterfalls[current_bond].loc[1,'Scheduled_Payment_' + current_bond] = np.nan
    
    final_df = pd.DataFrame(index=collateral_waterfall.index, columns=['remaining_cash'])
    
    #for k, v in collateral_waterfall.iterrows():
    for k in collateral_waterfall.index.values:
        rem_cash = np.float(collateral_waterfall.loc[k,'cash_flow'])
        # pay interest
        
        for i in range(len(bonds)):
            
            current_bond = bonds[i]['Bond']
            
            if k > 1:
                bond_waterfalls[current_bond].loc[k, 'Balance_' + current_bond] = \
                    bond_waterfalls[current_bond].loc[k-1, 'Balance_' + current_bond] - \
                    (bond_waterfalls[current_bond].loc[k-1, 'Cashflow_' + current_bond] - \
                    bond_waterfalls[current_bond].loc[k-1, 'Interest_' + current_bond])
            
            bond_waterfalls[current_bond].loc[k, 'Interest_' + current_bond] = round(-np.ppmt(
                                                 bond_waterfalls[current_bond].loc[k, 'Coupon_' + current_bond]/12,
                                                 1,
                                                 361-k,
                                                 bond_waterfalls[current_bond].loc[k, 'Balance_' + current_bond]),2)
            
            bond_waterfalls[current_bond].loc[k, 'Scheduled_Payment_' + current_bond] = round(-np.pmt(
                                                 bond_waterfalls[current_bond].loc[k, 'Coupon_' + current_bond]/12,
                                                 361-k,
                                                 bond_waterfalls[current_bond].loc[k, 'Balance_' + current_bond]),2)
            
            
            interest_cash_flow = min(
                np.float(bond_waterfalls[current_bond].loc[k, 'Interest_' + current_bond]), 
                np.float(rem_cash))
            
            bond_waterfalls[current_bond].loc[k, 'Cashflow_' + current_bond] = interest_cash_flow
            
            rem_cash -= interest_cash_flow
            
        # pay principal
        for i in range(len(bonds)):
            
            current_bond = bonds[i]['Bond']
            
            principal_cash_flow = min(
                np.float(bond_waterfalls[current_bond].loc[k, 'Balance_' + current_bond]), 
                np.float(rem_cash))
                    
            bond_waterfalls[current_bond].loc[k, 'Principal_' + current_bond] = principal_cash_flow
            bond_waterfalls[current_bond].loc[k, 'Cashflow_' + current_bond] += principal_cash_flow
            rem_cash -= principal_cash_flow
        
        final_df.loc[k,'remaining_cash'] = rem_cash
        
    for i in range(len(bonds)):
        current_bond = bonds[i]['Bond']
        final_df = final_df.merge(bond_waterfalls[current_bond], 
                                  left_index=True,
                                  right_index=True)

    return final_df
        

In [12]:
import xlwings as xw
wb = xw.Book()

sheet = wb.sheets(1)

In [46]:
import CMO_waterfall as cmo

struct = cmo.CMO(original_balance=initial_balance,
                             pass_thru_cpn=net_coupon,
                             wac=gross_coupon,
                             wam=maturity,
                             psa_speed=0,
                             servicing=servicing,
                             bonds=bonds)

print(((struct.waterfall.index.values * struct.waterfall['Cashflow_A']).sum() / 
    struct.waterfall['Cashflow_A'].sum()) / 12)

print(((struct.waterfall.index.values * struct.waterfall['Cashflow_B']).sum() / 
    struct.waterfall['Cashflow_B'].sum()) / 12)

print(((struct.waterfall.index.values * struct.waterfall['Cashflow_C']).sum() / 
    struct.waterfall['Cashflow_C'].sum()) / 12)


sheet.range('A1').expand().value = ""
sheet.range('A1').value = struct.waterfall[['Interest_A',
                                           'Principal_A',
                                           'Interest_B',
                                           'Principal_B',
                                           'Interest_C',
                                           'Principal_C',
                                           'remaining_cash']]

Initializing...
Creating collateral waterfall...
Producing CMO waterfall...
Merging waterfalls...
Done...
1.2104124301111552
3.8768455320743733
6.468043233772356


In [47]:
wb.sheets(2).range('A1').value = struct.collateral_waterfall